In [1]:
import os
path = '/Users/jeebanswain/Documents/Kaggle Challenges/Tech Geek Entity Recog'
os.chdir(path)
#os.listdir(path)

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth=2000
pd.options.display.max_rows= 1300

In [3]:
# Reading the files
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
samplesubmission = pd.read_csv('sample-submission.csv')

In [4]:
print train.shape
print test.shape
print samplesubmission.shape
#train.head()

(9850, 3)
(21070, 2)
(21070, 2)


In [5]:
# Total Unq strings
print len(train.StringToExtract.unique()) ##
dd = pd.DataFrame(train.StringToExtract.value_counts())
dd1 = dd.sort()
#sum(dd1[0:55][0])
# ddlsql43 = 3817 outof 9850
#print 3817.00/9850.00
print 100*118.00/9850.00 

1311
1.19796954315


In [6]:
## replace . by space 
import re
train['processed_desc'] = train.description.apply(lambda x : x.replace('.',' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace(':',' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace('"',' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace('/',' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace('[',' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace('(',' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace("\\r",' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace("\\n",' '))
train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace("'",' '))
#train['processed_desc'] = train.processed_desc.apply(lambda x : x.replace("-",'_'))

test['processed_desc'] = test.description.apply(lambda x : x.replace('.',' '))
test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace(':',' '))
test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace('"',' '))
test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace('/',' '))
#test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace('\\',' '))
test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace('[',' '))
test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace('(',' '))
test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace("'",' '))
#test['processed_desc'] = test.processed_desc.apply(lambda x : x.replace("-",'_'))
#train['ddl_keywords'] = train.description.apply(lambda x : str(list(set(re.findall(r'\bddlsql\w+', x)))))

In [7]:
dd = pd.DataFrame(train.StringToExtract.value_counts())
dd1 = dd.sort()
all_ips = dd1.index[0:57].tolist()

def get_ip(x):
    ip = re.findall( r'[0-9]+(?:\.[0-9]+){3}', x )
    if(len(ip)>0):
        ip=ip[0]
        
        return ip      
    else:
        return 'not an ip'

    
train['present_ip'] = train.description.apply(get_ip)
test['present_ip'] = test.description.apply(get_ip)

def is_ip(x):
    if(x in (all_ips)):
        return 1
    else:
        return 0

train['is_ip'] = train.StringToExtract.apply(is_ip )
#test['is_ip'] = test.StringToExtract.apply(is_ip)

In [10]:
#in_correct_category ==
#print 100*test[test.is_ip==1].shape[0]/21070.0
#print 100*test[test.final_string=='will'].shape[0]/21070.0
#print 100*test[test.final_string=='service'].shape[0]/21070.0
#print 100*test[test.final_string=='internal'].shape[0]/21070.0
#test.shape

In [ ]:
def get_service_entity(row):
    indx_1 = row.processed_desc.find('criticalservicedown\r\r\n service')
    indx_2 = row.processed_desc.find('criticalservicedown\r\r\nseverity')
    
    indx_3 = row.processed_desc.find('\r\r\ncisco prime lm admin')
    indx_4 = row.processed_desc.find('cisco callnageram cisco ip phone')
    
    #ip_in_ips = row.present_ip in (all_ips)
    
    if( ((indx_1<>-1) | (indx_2<>-1))  ):
        if(((indx_3<>-1) | (indx_4<>-1))):
            return 0
        else:
            return 1
    else:
        return 0
    
train['service_keywords'] = train.apply(get_service_entity,axis=1)
test['service_keywords'] = test.apply(get_service_entity,axis=1)

In [ ]:
#print error_all[error_all.is_ip==1].shape
#error_all.StringToExtract.value_counts()
#error_all[error_all.is_ip==1][['StringToExtract','description']]

def ip_keywords(row):
    indx_1 = row.processed_desc.find('ddl-oionr-01')
    indx_2 = row.processed_desc.find('ddl-msg-mgmt03')
    indx_3 = row.processed_desc.find('esc-sim-01')
    indx_4 = row.processed_desc.find('ddl-sim-02')
    indx_5 = row.processed_desc.find('was-sanmgt-01')
    indx_6 = row.processed_desc.find('dimensiondataservices')
    
    ip_in_ips = row.present_ip in (all_ips)
    if(( (indx_1<>-1)  | (indx_2<>-1) | (indx_3<>-1) |
        (indx_4<>-1) | (indx_5<>-1) ) & (ip_in_ips == True) ):
        
        if( (indx_6<>-1)):
            return 0
        else:
            return 1
    #if((row.present_ip<>'not an ip') & (row.ddl_3_keywords in(['internal','error'])) ):
        #& (row.ddl_3_keywords in(['internal','error','rom-ug-dsw1-01'])) ):
        #if( (ip_in_ips) ):
        return 1
    else:
        return 0
    
train['ip_keywords'] = train.apply(ip_keywords,axis=1)
test['ip_keywords'] = test.apply(ip_keywords,axis=1)
print sum(train['ip_keywords'])

In [16]:
def get_all_internal(x):
    indx1 = x.find('zone internal')
    indx2 = x.find('domain:internal')
    if((indx1<>-1) | (indx2<>-1) ):
        return 1
    else:
        return 0
    
train['is_internal'] = train.description.apply(get_all_internal)
test['is_internal'] = test.description.apply(get_all_internal)
print train[train.is_internal==1].shape
print test[test.is_internal==1].shape

(5, 13)
(9, 11)


In [18]:

#train.head()
prefixes = tuple(['ddl-','ddls','esc-','nyc','fra-',
                  'hkg','ams-','asi-','au-','del-','esc-',
                  'hdr-','hkg-','ldv-','par-','pra-','rom-','wcms-'
                 'lpp-','was-','lpdb','muc-','mos-','esap',
                 'syd-','war-','hkgs','bru-','escs','buc-','mad-',
                 'ist-','lux-','bei-','shn-','asi-','per-','lpld',
                 'cas-','sin-','esbd','bar-','ddl_','bkk-','sap-',
                 'mil-','dub-','us-','dus-','doh-','cn-','eur-',
                 'ind-','jp-','tok-','dus_','doh_','ce_','nje-',
                 'aud-','lst-','riy-'])


prefixes = ['bru_', 'dus-', 'ldv-', 'nyc-', 'mos-', 'bei-', 'fra-', 'del-',
       'lpdb', 'lpp-', 'dub-', 'war-', 'hkgs', 'hdr-', 'eur-', 'shn-',
       'ce_', 'aud-', 'riy-', 'buc-', 'cas-', 'esap', 'wcms-lpp-', 'mad-',
       'blr-', 'lpld', 'ddl-', 'hkg-','ind-', 'nyc', 'rom-', 'par-',
       'tok-', 'hkg_', 'esbd', 'doh_', 'jp-', 'ddl_', 'rom_', 'doh-',
       'esc-', 'asi-', 'sin_', 'bkk-', 'seo-', 'was-', 'buc_', 'riy_',
       'lst-', 'mil-', 'bar-', 'hkg', 'escs', 'sap-', 'ams-', 'cn-',
       'syd-', 'nje-', 'lux-', 'us-', 'ddls', 'au-', 'ist-', 'per-',
       'dus_', 'bru-', 'sin-', 'nyc_', 'muc-', 'fra_', 'mos_', 'bei_',
       'pra-','grdp','esc-top-01']



all_entities = list(train.StringToExtract)


def pattern_matching(x):
    
    f1 = re.findall('([^ \r\n]+) controlnet([\r\n]| |$)', x, re.IGNORECASE)
    f2 = re.findall('([^ \r\n]+) intranet([\r\n]| |$)', x, re.IGNORECASE)
    f3 = re.findall('([^ \r\n]+) internal([\r\n]| |$)', x, re.IGNORECASE)
    ip = re.findall( r'[0-9]+(?:\.[0-9]+){3}', x )

    if(len(ip)>0):
        ip = list(ip[0] + '  roalswinds')
    
    
    f = f1+f2+f3+ip
    
    new_prefix = []
    
    for result in f:
        new_prefix.append(result[0])
        
    
    prefixes_1 =  prefixes + new_prefix
    
    
    entity = [ t for t in x.split() if ( (t.startswith(tuple(prefixes_1))) and (t in all_entities) )]
    
    #entity = set((entity))
    #entity = list( entity & all_entities)
   
    if(len(entity)>0):
        entity = list((entity))[0] 
    else:
        entity = 'not found'
        #entity = [ t for t in x.split() if (t.count('-')==2)][0]

    return entity

error_prefix_1 = ['esc-top-01','alumni','hkg-mb-13','hdr-mb-15','epinux10',
               'gpmszmaas_ptl2','lsldap01','lsldap02','lpap03','lpap02z0','lpap02z1',
               'serverfqdn=ddl-mb-08','ddl-mb-02','ddl-dc-03','esdb05d0','lpap02','lpap01',
               'serverfqdn=hkg-mb-14','dub-prn-01','esdb05','fra-dc-03',
               'hyppv11','was-mb-11','esdb03','sgddl_esc-spr1fast-','lpinvm01','lpmg01',
               'ddl-edg-v02','fra-dcr-02','gpmszic3','americas_was_vtl_adic_scalar_i2k_lt4_l01',
               'time-u','nyc-spr2web-01','dub-dc-03','esc-mb-05','lsldap05','nyc-mb-09','americas_was_adic',
                'hkg-sql-cl1v01','lpinux10','asia_hkp_vtl_adic_scalar_i2k_lt4_l01','gpmszfd1','hkg-ev-14',
                'esc-ev-11','au-per-06a-stcp-01','ldtpocs_ddldon','rom-ug-dsw1-01','buc_dsw-01','esc-ev-10',
                'war-05-dsw-02','au-per-06a-stap-01','blr-nwa-sw2','buc_dsw-02','war-04-dsw-01',
                'hkg-dcr-02','onlineservices','dc=internal'       
               ]

error_prefix_2 = ['error','author','connect','internal']

error_prefix_0 = ['serverfqdn=hkg-mb-14.internal','hkg-mb-14.internal','ddl-dc-06.internal',
                 'hdr-mb-16.internal','nyc-bes-29.internal','nyc-bes-30.internal','hkg-bes-27.internal',
                  'bru-dc-02.internal','esc-mb-03.internal','esc-mb-04.internal','ddl-bnt-01.internal'
                  ,'ddl-ch-v03.internal','dub-dc-03.internal','hkg-ev-14.internal','fra-dcr-02.ecnahcdroffilc',
                  'dc=internal.ecnahcdroffilc.net','time.pm.ecnahcdroffilc.com','spr1.intranet.ecnahcdroffilc',
                  'ddl-dc-01.internal','ddl-sql-cl11an1.internal','hdr-mb-15.internal','esc-bes-20.internal','bkk-dc-02.internal'
                 ]

def errored_pattern_0(x):
    x = x.replace("//",' ')
    x = x.replace(",",' ')
    x = x.replace("/",' ')
    x =x.replace("'",'')
    x =x.replace(":",' ')
    x =x.replace(";",' ')
   
    
    new_prefix = []
    f1 = re.findall('([^ \r\n]+) internal.ecnahcdroffilc.net([\r\n]| |$)', x, re.IGNORECASE)
    f2 = re.findall('([^ \r\n]+) .ecnahcdroffilc.com([\r\n]| |$)', x, re.IGNORECASE)
    f3 = re.findall('([^ \r\n]+) .internal.ecnahcdroffilc.net([\r\n]| |$)', x, re.IGNORECASE)
    f_new = f1+f2+f3

    
    for result in f_new:
        new_prefix.append(result[0])  
    
    error_prefix_0_1 =  error_prefix_0 + new_prefix
    
    entity = [ t for t in x.split() if ((t.startswith(tuple(error_prefix_0_1))) )  ]
    
    entities = []
    if(len(entity)>0):
        #entity = entity[0]
        #entity = entity.split(';')
        #entity = entity.replace('.',' ')[0]
        #entity = entity.replace("'",' ')
        for ent in entity:
            ent = ent.replace('internal','')
            ent = ent.replace('ecnahcdroffilc','')
            ent = ent.replace('net','')
            ent = ent.replace(".",'')
            ent = ent.replace("dc=",'dc=internal')
            ent = ent.replace("pm",'')
            ent = ent.replace("com",'')
            ent = ent.replace("intra",'')
            ent = ent.replace("80",'')
            entities.append(ent)
        #entity = entity.strip()
        
        entity = [entity for entity in entities if (entity in all_entities)]
        if(len(entity)>0):
            entity =entity[0]
        else:
            entity = 'not found'
            
    else:
        entity = 'not found'
    
    return entity


def errored_pattern_1(x):
    entity = [ t for t in x.split() if ((t.startswith(tuple(error_prefix_1))) and (t in all_entities))]
    if(len(entity)>0):
        #entity = max(entity,key=entity.count)
        entity = entity[0]
        #entity = entity.replace(' internal','')
    else:
        entity = 'not found'
        
        
    return entity

def errored_pattern_2(x):
    entity = [ t for t in x.split() if ((t.startswith(tuple(error_prefix_2))) and (t in all_entities))]
    
    if(len(entity)>0):
        entity = entity[0]
    else:
        entity = 'not found'
        
        
    return entity



train['service_keywords'] = train.apply(get_service_entity,axis=1)
test['service_keywords'] = test.apply(get_service_entity,axis=1)

train['ip_keywords'] = train.apply(ip_keywords,axis=1)
test['ip_keywords'] = test.apply(ip_keywords,axis=1)



train['ddl_0_keywords'] = train.description.apply(errored_pattern_0)
train['ddl_1_keywords'] = train.processed_desc.apply(errored_pattern_1)
train['ddl_1_keywords'] = np.where(train.ddl_0_keywords=='not found',train.ddl_1_keywords,train.ddl_0_keywords)
train['ddl_2_keywords'] = train.processed_desc.apply(pattern_matching)
train['ddl_1_keywords'] = np.where(train.ddl_1_keywords=='not found',train.ddl_2_keywords,train.ddl_1_keywords)
train['ddl_3_keywords'] = train.processed_desc.apply(errored_pattern_2)
train['ddl_1_keywords'] = np.where(train.ddl_1_keywords=='not found',train.ddl_3_keywords,train.ddl_1_keywords)

train['ddl_1_keywords'] = train.ddl_1_keywords.apply(lambda x : x.strip())


test['ddl_0_keywords'] = test.description.apply(errored_pattern_0)
test['ddl_1_keywords'] = test.processed_desc.apply(errored_pattern_1)
test['ddl_1_keywords'] = np.where(test.ddl_0_keywords=='not found',test.ddl_1_keywords,test.ddl_0_keywords)
test['ddl_2_keywords'] = test.processed_desc.apply(pattern_matching)
test['ddl_1_keywords'] = np.where(test.ddl_1_keywords=='not found',test.ddl_2_keywords,test.ddl_1_keywords)
test['ddl_3_keywords'] = test.processed_desc.apply(errored_pattern_2)
test['ddl_1_keywords'] = np.where(test.ddl_1_keywords=='not found',test.ddl_3_keywords,test.ddl_1_keywords)
test['ddl_1_keywords'] = test.ddl_1_keywords.apply(lambda x : x.strip())

train['is_internal'] = train.description.apply(get_all_internal)
test['is_internal'] = test.description.apply(get_all_internal)



#train['service_keywords'] = train.apply(get_service_entity,axis=1)
#test['service_keywords'] = test.apply(get_service_entity,axis=1)

train['ddl_1_keywords'] = np.where( (train.service_keywords==1), 'service',train.ddl_1_keywords )
test['ddl_1_keywords'] = np.where(  (test.service_keywords==1), 'service',test.ddl_1_keywords )

train['ddl_1_keywords'] = np.where((train.ddl_1_keywords<>'service') & (train.ip_keywords==1), train.present_ip,train.ddl_1_keywords )
test['ddl_1_keywords'] = np.where((test.ddl_1_keywords<>'service') & (test.ip_keywords==1), test.present_ip,test.ddl_1_keywords )

train['ddl_1_keywords'] = np.where( (train.is_internal==1), 'internal',train.ddl_1_keywords )
test['ddl_1_keywords'] = np.where(  (test.is_internal==1), 'internal',test.ddl_1_keywords )


train['is_matched'] = np.where(train.StringToExtract==train.ddl_1_keywords,1,0)

print sum(np.where(train.StringToExtract==train.ddl_1_keywords,1,0))
print train.is_matched.mean()

error_all = train[(train.is_matched==0)]
print error_all.shape
error = train[(train.is_matched==0) & (train.is_ip==0) & (train.StringToExtract<>'service') & (train.StringToExtract<>'will')]
#print train[train.StringToExtract=='service'].shape
print error.shape
print 1-(error.shape[0]/9850.00)

9845
0.999492385787
(5, 14)
(0, 14)
1.0


In [19]:
best_sub = pd.read_csv('best_sub.csv')
test['final_string'] = np.where(best_sub.StringToExtract.isin(['will','internal']),best_sub.StringToExtract,test.ddl_1_keywords)
samplesubmission.StringToExtract = test.ddl_1_keywords
samplesubmission.to_csv('Final_with_Regex.csv',index=False)
samplesubmission.head()

,id,StringToExtract
0,1,frapub2
1,2,lpap02z1
2,4,ddlsql43
3,5,ddl-ubs-28-ca-asw1
4,6,ddlsql43


In [318]:
test_entity = set(test.final_string.unique())
train_entity = set(train.StringToExtract.unique())
diffr_entity = list(train_entity - test_entity)
#len(test_entity)
#print len(diffr_entity)
train_differed_entity = train[train.StringToExtract.isin(diffr_entity)]
#train_differed_entity.head()
diffr_entity[0:10]

['nyc-cca4-ddc2',
 'psel1661',
 'lpp-csidx-01',
 'ddl-dml-01',
 'hdr-sql-cl3',
 'pprg646',
 'fra-print-01',
 'lsldap05',
 'doh_asw1_u',
 'pdxb1470']

In [371]:
for different_entity in diffr_entity[50:100]:
    def quick_chk(x):
        #different_entity = 'hkg-spr3crwl-02'
        indx = x.find(different_entity)
        #indx = x.find(different_entity)
        if(indx<>-1):
            return 1
        else:
            return 0
    train['chk'] = train.processed_desc.apply(quick_chk)
    test['chk'] = test.processed_desc.apply(quick_chk)
    #print sum(train['chk'])
    print different_entity
    print sum(test['chk'])


psel1623
0
hkg-da-11
0
ddl-sql-cl11dtc
0
psel1436
0
mad-dms-01
0
ddlsql127
0
ddl-com-120
0
hdr-bes-27dr
0
doh-vpns-01
0
doh-vpns-02
0
escsql120
0
par-apps-02
0
nyc-sql-07
0
ddl-ubs-03-cb-asw1
0
del-s1-f3-asw4
0
time-u
0
ddl-hpcm-02
0
ddlsql112
0
riy-vpns-02
0
ldv-gpms-03
0
ddl-wallb-01
0
ldv-spdcsql-02
0
dubgen4
0
esc-ch-v02
0
ppss
1
ddlsql16
0
ddlsql19
0
ddl-web-01
0
ddl-web-02
0
pprg1438
0
del-s1-f3-r2
0
ddl-ch-v03
1
pvsupport
0
escsqlcca41
0
ddl-sql-cl11
10
del-f2-asw1
0
6.1.7221.0
0
ddl-ubs-03-cd-asw1
0
ddlsql23
0
del-s2-f4-asw4
0
del-s2-f4-asw5
0
pra-5th-dsw1-01
0
del-s2-f4-asw2
0
del-s2-f4-asw3
0
fra-dms-03v
0
hdr-gmm-01
0
buc-hpcm-01
0
10.48.103.111
0
ddl-epo-02
0


In [372]:
different_entity = 'ddl-ch-v03'
def quick_chk(x):
        indx = x.find(different_entity)
        #indx = x.find(different_entity)
        if(indx<>-1):
            return 1
        else:
            return 0
train['chk'] = train.processed_desc.apply(quick_chk)
test['chk'] = test.processed_desc.apply(quick_chk)
#print sum(train['chk'])
print different_entity
print sum(test['chk'])

ddl-ch-v03
1


In [373]:
train[train.StringToExtract== 'ddl-ch-v03']

,id,StringToExtract,description,processed_desc,present_ip,is_ip,service_keywords,ddl_0_keywords,ddl_1_keywords,ddl_2_keywords,ddl_3_keywords,is_matched,ip_keywords,is_will,is_internal,chk
3839,12127,ddl-ch-v03,"tcp connectivity against local server - address book failure (nspi) the test step getprofiledetails running over protocol 'ncacn_ip_tcp' issued from test user '/o=ifrdoflc ccaehn/ou=ddldon/cn=recipients/cn=extest_25f912293bca4' failed with error: '-2147467259(unspecified error)'. the detailed information of the running instance context: 'user = 'extest_25f912293bca4', server = 'ddl-ch-v03.internal.ecnahcdroffilc.net', usehttp = 'false''. the exception: 'fotrscomi.eehaxgnc.monitoring.nspioperationexception: the nspi operation failed. operation = '<queryhomemdb>b__8', returnvalue = '-2147467259', server = 'ddl-ch-v03.internal.ecnahcdroffilc.net', user = '/o=ifrdoflc ccaehn/ou=ddldon/cn=recipients/cn=extest_25f912293bca4', .\r\r\n at fotrscomi.eehaxgnc.monitoring.nspiclientwrapper.nspicall(func`1 nspicall)\r\r\n at fotrscomi.eehaxgnc.monitoring.nspiclientwrapper.queryhomemdb(string primarysmtpaddress, legacydn& homemdb, legacydn& userlegacydn)\r\r\n at fotrscomi.eehaxgnc.monitoring.addressbooktask.getprofile()'\r\r\nverbose: target site = 'ecnahcdroffilc.net/configuration/sites/ddldon'\r\r\nverbose: target domain = 'internal.ecnahcdroffilc.net'\r\r\nverbose: target user = 'extest_25f912293bca4@internal.ecnahcdroffilc.net'\r\r\nverbose: based on rpctesttype, the cmdlet needs to dynamically look up the endpoint that uses this server as reference point.\r\r\nverbose: target client access server = 'ddl-ch-v03.internal.ecnahcdroffilc.net'\r\r\nverbose: rpc endpoint = 'ddl-ch-v03.internal.ecnahcdroffilc.net'\r\r\nverbose: rpc endpoint = 'ddl-ch-v03.internal.ecnahcdroffilc.net'\r\r\nverbose: rpc endpoint = 'ddl-ch-v03.internal.ecnahcdroffilc.net'\r\r\nverbose: using connection parameters : 'internal.ecnahcdroffilc.net\extest_25f912293bca4: rfri/tcp-ip, [ddl-ch-v03.internal.ecnahcdroffilc.net/negotiate]'\r\r\nverbose: mloxbia = 'internal.ecnahcdroffilc.net\extest_25f912293bca4', owner = '/o=ifrdoflc ccaehn/ou=ddldon/cn=recipients/cn=extest_25f912293bca4'\r\r\nverbose...","tcp connectivity against local server - address book failure nspi) the test step getprofiledetails running over protocol ncacn_ip_tcp issued from test user o=ifrdoflc ccaehn ou=ddldon cn=recipients cn=extest_25f912293bca4 failed with error -2147467259 unspecified error) the detailed information of the running instance context user = extest_25f912293bca4 , server = ddl-ch-v03 internal ecnahcdroffilc net , usehttp = false the exception fotrscomi eehaxgnc monitoring nspioperationexception the nspi operation failed operation = <queryhomemdb>b__8 , returnvalue = -2147467259 , server = ddl-ch-v03 internal ecnahcdroffilc net , user = o=ifrdoflc ccaehn ou=ddldon cn=recipients cn=extest_25f912293bca4 , \r\r\n at fotrscomi eehaxgnc monitoring nspiclientwrapper nspicall func`1 nspicall)\r\r\n at fotrscomi eehaxgnc monitoring nspiclientwrapper queryhomemdb string primarysmtpaddress, legacydn& homemdb, legacydn& userlegacydn)\r\r\n at fotrscomi eehaxgnc monitoring addressbooktask getprofile ) \r\r\nverbose target site = ecnahcdroffilc net configuration sites ddldon \r\r\nverbose target domain = internal ecnahcdroffilc net \r\r\nverbose target user = extest_25f912293bca4@internal ecnahcdroffilc net \r\r\nverbose based on rpctesttype, the cmdlet needs to dynamically look up the endpoint that uses this server as reference point \r\r\nverbose target client access server = ddl-ch-v03 internal ecnahcdroffilc net \r\r\nverbose rpc endpoint = ddl-ch-v03 internal ecnahcdroffilc net \r\r\nverbose rpc endpoint = ddl-ch-v03 internal ecnahcdroffilc net \r\r\nverbose rpc endpoint = ddl-ch-v03 internal ecnahcdroffilc net \r\r\nverbose using connection parameters internal ecnahcdroffilc net\extest_25f912293bca4 rfri tcp-ip, ddl-ch-v03 internal ecnahcdroffilc net negotiate] \r\r\nverbose mlox

In [374]:
test[test.chk==1]

,id,description,processed_desc,present_ip,service_keywords,ddl_0_keywords,ddl_1_keywords,ddl_2_keywords,ddl_3_keywords,ip_keywords,best_sub_String,is_ip_best,is_will,is_internal,final_string,chk
59,87,"monitoring failed to run the test-mrsteahhl cmdlet. an active directory error 0x51 occurred when trying to check the suitability of server 'ddl-dc-06.internal.ecnahcdroffilc.net'. error: 'active directory response: the ldap server is unavailable.'\r\r\n\r\r\ndiagnostic command: ""test-mrsteahhl -monitoringcontext:$true""\r\r\neventsourcename: mseehaxgnc monitoring mrsteahhl\r\r\n\r\r\n\r\r\n knowledge: http://go.fotrscomi.com/fwlink/?linkid=67336&id=80d898e6-58fe-45e1-9c04-0ba31357b532\r\r\n\r\r\n computer: ddl-ch-v03.internal.ecnahcdroffilc.net\r\r\n\r\r\n impacted entities (0): fotrscomi.eehaxgnc.2010.clientaccessmloxbiareplicationservice:ddl-ch-v03.internal.ecnahcdroffilc.net;ddl-ch-v03 (client access) - ddldon;mloxbia replication service - ddl-ch-v03 (client access) - ddldon ddl-ch-v03.internal.ecnahcdroffilc.net;ddl-ch-v03 (client access) - ddldon false 255 1 1 na na alert system 3-25-2017 6:38:00 am 3-25-2017 6:43:58 am 358 ddldonmocs",monitoring failed to run the test-mrsteahhl cmdlet an active directory error 0x51 occurred when trying to check the suitability of server ddl-dc-06 internal ecnahcdroffilc net error active directory response the ldap server is unavailable \r\r\n\r\r\ndiagnostic command test-mrsteahhl -monitoringcontext $true \r\r\neventsourcename mseehaxgnc monitoring mrsteahhl\r\r\n\r\r\n\r\r\n knowledge http go fotrscomi com fwlink ?linkid=67336&id=80d898e6-58fe-45e1-9c04-0ba31357b532\r\r\n\r\r\n computer ddl-ch-v03 internal ecnahcdroffilc net\r\r\n\r\r\n impacted entities 0) fotrscomi eehaxgnc 2010 clientaccessmloxbiareplicationservice ddl-ch-v03 internal ecnahcdroffilc net;ddl-ch-v03 client access) - ddldon;mloxbia replication service - ddl-ch-v03 client access) - ddldon ddl-ch-v03 internal ecnahcdroffilc net;ddl-ch-v03 client access) - ddldon false 255 1 1 na na alert system 3-25-2017 6 38 00 am 3-25-2017 6 43 58 am 358 ddldonmocs,not an ip,0,ddl-dc-06,ddl-dc-06,ddl-dc-06,error,0,ddl-dc-06,0,0,0,ddl-dc-06,1


In [ ]:
## IDs to be corrected : 6045 - hkg-spr3crwl-02
# 
test[test]

In [263]:
will_list =['ddl-mocsgw-03',
 'ddl-sim-02',
 'esc-sim-01',
 'hdr-sanmgt-01',
 'hkg-mocsgw-01',
 'hkg-sanmgt-01',
 'nyc-sanmgt-01',
 'was-sanmgt-01']


def get_all_will(x):
    #indx1 = x.find('kb4013867')
    indx2 = x.find('computer:ddl-web')
    indx3 = x.find('computer:ddl-sanmgt')
    indx4 = x.find('computer:nyc-mocsgw')
    indx5 = x.find('3l1574l6')
    #indx6 = x.find('was-sanmgt-01')
    #indx7 = x.find('kb4013867')
    #indx8 = x.find('kb4013867')
    if(  ((indx2<>-1) | (indx3<>-1) | (indx4 <>-1) | (indx5<>-1))):
        return 1
    else:
        return 0
    
train['is_will'] = train.description.apply(get_all_will)
test['is_will'] = test.description.apply(get_all_will)
#error_all.processed_desc.apply(get_all_will)
print train[train.is_will==1].shape
print test[test.is_will==1].shape

(6, 14)
(3, 13)


In [282]:
def get_all_internal(x):
    indx1 = x.find('zone internal')
    indx2 = x.find('domain:internal')
    if((indx1<>-1) | (indx2<>-1) ):
        return 1
    else:
        return 0
    
train['is_internal'] = train.description.apply(get_all_internal)
test['is_internal'] = test.description.apply(get_all_internal)
print train[train.is_internal==1].shape
print test[test.is_internal==1].shape

(5, 15)
(9, 14)


In [293]:
#test[test.is_internal==1].id.unique()

In [284]:
#train[train.StringToExtract=='internal']
#[test.is_will==1]

In [256]:
#train[train.StringToExtract=='will']
#train.description[477].find('kb4013867')

In [196]:
#train[train.is_will==1]
#[('esc-cvbkpcs-01', ' '), ('esc-cvbkpcs-01', ' ')][0][0]

In [700]:
#train[(train.ddl_1_keywords=='service')]
# \r\r\ncisco prime lm admin
# cisco callnageram cisco ip phone

In [659]:
#100*148.0/train.shape[0]

In [651]:
99.7874 + 100*24.0/test.shape[0]

99.9013060275273

In [654]:
error_all

In [146]:
samplesubmission.StringToExtract = test.ddl_1_keywords
#samplesubmission.StringToExtract 
samplesubmission.to_csv('only_regex_no_will_intrenal.csv',index=False)
samplesubmission.head(10)

,id,StringToExtract
0,1,frapub2
1,2,lpap02z1
2,4,ddlsql43
3,5,ddl-ubs-28-ca-asw1
4,6,ddlsql43
5,8,ddlsql43
6,9,frapub2
7,10,ddlsql43
8,11,esc-spr4web-01
9,12,ddlsql43


In [ ]:
# IPS = 120 , service = 19 , will = 5 internal = 4

In [630]:
train[train.StringToExtract==train.present_ip].shape

(120, 14)

In [14]:
#print error_all[error_all.is_ip==1].shape
#error_all.StringToExtract.value_counts()
#error_all[error_all.is_ip==1][['StringToExtract','description']]

def ip_keywords(row):
    indx_1 = row.processed_desc.find('ddl-oionr-01')
    indx_2 = row.processed_desc.find('ddl-msg-mgmt03')
    indx_3 = row.processed_desc.find('esc-sim-01')
    indx_4 = row.processed_desc.find('ddl-sim-02')
    indx_5 = row.processed_desc.find('was-sanmgt-01')
    indx_6 = row.processed_desc.find('dimensiondataservices')
    
    ip_in_ips = row.present_ip in (all_ips)
    if(( (indx_1<>-1)  | (indx_2<>-1) | (indx_3<>-1) |
        (indx_4<>-1) | (indx_5<>-1) ) & (ip_in_ips == True) ):
        
        if( (indx_6<>-1)):
            return 0
        else:
            return 1
    #if((row.present_ip<>'not an ip') & (row.ddl_3_keywords in(['internal','error'])) ):
        #& (row.ddl_3_keywords in(['internal','error','rom-ug-dsw1-01'])) ):
        #if( (ip_in_ips) ):
        return 1
    else:
        return 0
    
train['ip_keywords'] = train.apply(ip_keywords,axis=1)
test['ip_keywords'] = test.apply(ip_keywords,axis=1)
print sum(train['ip_keywords'])

139


In [144]:
train[(train.ip_keywords==1) & (train.service_keywords<>1) & (train.is_ip<>1)]

,id,StringToExtract,description,processed_desc,present_ip,is_ip,service_keywords,ddl_0_keywords,ddl_1_keywords,ddl_2_keywords,ddl_3_keywords,is_matched,ip_keywords


In [119]:
#train[train.ddl_3_keywords.isin(['error','internal'])].shape

In [121]:
#train[train.services_keywords==1]
#test[test.services_keywords==1].shape
#train[train.StringToExtract=='services']

AttributeError: 'DataFrame' object has no attribute 'services_keywords'

In [80]:
train[train.StringToExtract=='error'].shape

(3, 13)

In [79]:
train[(train.ip_keywords==1) & (train.StringToExtract<>'service')].shape

(0, 13)

In [27]:
print train[(train.ip_keywords==1) & (train.service_keywords==0)].shape
print test[(test.ip_keywords==1) & (test.service_keywords==0)].shape
print test[test.ip_keywords==1].shape

(121, 13)
(202, 10)
(245, 10)


In [34]:
#test[(test.ip_keywords==1) & (test.service_keywords==0)].head()
#train[train.present_ip<>'not an ip'].head(50)
#train[(train.ip_keywords==0) & (train.is_ip==1)].head()
#train[train.is_ip==1]
# internal ecnahcdroffilc net na false 255
# internal ecnahcdroffilc net false 255
# internal ecnahcdroffilc net true 255
#train[train.present_ip<>'not an ip'].head(100)

In [849]:
#train[(train.ip_keywords==1) & (train.service_keywords==0)].ddl_1_keywords.value_counts()
# nyc-fw-05
#train[train.StringToExtract.isin (all_ips)]

In [12]:
def get_service_entity(row):
    indx_1 = row.processed_desc.find('criticalservicedown\r\r\n service')
    indx_2 = row.processed_desc.find('criticalservicedown\r\r\nseverity')
    
    indx_3 = row.processed_desc.find('\r\r\ncisco prime lm admin')
    indx_4 = row.processed_desc.find('cisco callnageram cisco ip phone')
    
    #ip_in_ips = row.present_ip in (all_ips)
    
    if( ((indx_1<>-1) | (indx_2<>-1))  ):
        if(((indx_3<>-1) | (indx_4<>-1))):
            return 0
        else:
            return 1
    else:
        return 0
    
train['service_keywords'] = train.apply(get_service_entity,axis=1)
test['service_keywords'] = test.apply(get_service_entity,axis=1)

In [15]:
train[(train.service_keywords==1) & (train.StringToExtract=='service')].shape

(19, 7)

In [735]:
print train[train.service_keywords==1].shape
print test[test.service_keywords==1].shape

(19, 14)
(43, 15)


In [559]:
#train[(train.service_keywords==1)]
test[(test.ip_keywords==1) & (test.service_keywords==0)].shape

(198, 14)

In [56]:
#print train[(train.ip_keywords==1) | (train.is_ip==1)].StringToExtract.value_counts()
print test[test.ip_keywords==1].shape
print test[((test.ip_keywords==1) | (test.is_ip==1)) & (test.final_string=='service')].shape
#train[(train.ip_keywords==1)]
#train[train.StringToExtract=='service']

(245, 12)


AttributeError: 'DataFrame' object has no attribute 'is_ip'

In [421]:
#train[train.StringToExtract=='10.200.3.68'].shape

In [422]:
#train[train.StringToExtract=='service']

In [428]:
test[(test.final_string=='service')].shape

(43, 13)

In [328]:
print train[train.StringToExtract=='will'].shape
print test[test.final_string=='will'].shape

(5, 11)
(8, 12)


In [330]:
#train[train.StringToExtract=='will']

In [161]:

bf4_1, term, after = x.partition('internal ecnahcdroffilc')
bf4_1 = bf4_1.split()[-1]
bf4_2, term, after = x.partition('ecnahcdroffilc com')
bf4_2 = bf4_2.split()[-1]

f_new = bf4_1 + bf4_2
x

'By the Parking here is horrible, this shop sucks.'

In [151]:
#error.processed_desc[3839]
x = line 
bf4_1, term, after = x.partition('internal ecnahcdroffilc')
bf4_1 = bf4_1.split()[-1]
bf4_2, term, after = x.partition('ecnahcdroffilc com')
bf4_2 = bf4_2.split()[-1]
f_new = bf4_1 + bf4_2
new_prefix = []
for result in f_new:
    new_prefix.append(result[0])


In [773]:
#test[test.final_string=='will']
train[train.StringToExtract=='ddl-oionr-01'].shape

(0, 14)

In [850]:
#train[train.is_ip==1]

In [616]:
### Error from regex == hkg-ev-14 was as hkg-mb-13
# time
# 
#samp ="dns 2008/r2 troubleshoot ad ds and retry adding zone alert the dns server timed out attempting an active directory service operation on dc=luxesx02ilo,dc=internal.ecnahcdroffilc.net,cn=fotrscomidns,dc=domaindnszones,dc=internal,dc=ecnahcdroffilc,dc=net. check active directory to see that it is functioning properly. the event data contains the error.\r\r\n fotrscomi.windows.dnsserver.library.server:lux-dc-02.internal.ecnahcdroffilc.net;lux-dc-02.internal.ecnahcdroffilc.net lux-dc-02.internal.ecnahcdroffilc.net true 255 2 1 na na configurationteahhl system 3-26-2017 2:07:33 am 3-26-2017 2:13:51 am 377 ddldonmocs"
#errored_pattern_0(samp)

'not found'

In [ ]:
#train[train.StringToExtract==hkg-mb-14.internal]

In [770]:
#error
3600
5042
5506
9596
12341
13308
14907
18553
20009
21647
23079

23079

In [55]:
best_sub = pd.read_csv('best_sub.csv')
#all_test_ips = tra
dd2 = pd.DataFrame(best_sub.StringToExtract.value_counts())
dd2 = dd2.sort()
#dd2
all_ips_test = dd2.index[1:55].tolist()

test['best_sub_String'] = best_sub.StringToExtract
test['is_ip_best'] = np.where((test.best_sub_String.isin(all_ips_test) ) | ( (test.service_keywords==0) & (test.ip_keywords==1)),1,0)
print sum(test.is_ip_best)
#test['others'] = np.where ((test.is_ip_best==1),1,0)
#print sum(np.where ((test.best_sub_String.isin(['will','internal']) | (test.is_ip_best==1)),1,0))
#test['final_string'] = np.where(test.others==1,best_sub.StringToExtract,test.ddl_1_keywords)

202


In [886]:
#test[(test.is_ip_best==1) | (test.ip_keywords==1)].shape
sum(np.where((test.is_ip_best==1) | (test.ip_keywords==1),1,0))

207

In [770]:
#sum(np.where(test.best_sub_String.isin(all_ips_test),1,0)

In [603]:
#train[train.StringToExtract=='services'].shape

In [614]:
#train[train.StringToExtract=='services']

In [764]:
manual_correction_id = [87,178,3935,4142,4738,10930,12315,19132,21473,21903,23554,25332]
test['final_string']  = np.where(test['id'].isin(manual_correction_id),test.xgb_String,test['final_string'])

In [889]:
samplesubmission.StringToExtract = test.final_string
#samplesubmission.StringToExtract 
samplesubmission.to_csv('all_regex_ip_rgex_try1.csv',index=False)
samplesubmission.head(10)

,id,StringToExtract
0,1,frapub2
1,2,lpap02z1
2,4,ddlsql43
3,5,ddl-ubs-28-ca-asw1
4,6,ddlsql43
5,8,ddlsql43
6,9,frapub2
7,10,ddlsql43
8,11,esc-spr4web-01
9,12,ddlsql43


In [767]:
#test[test['id'].isin(manual_correction_id)].head()

In [ ]:
## keywords to be replaced with will
{'ddl-mocsgw-03',
 'ddl-sim-02',
 'esc-sim-01',
 'hdr-sanmgt-01',
 'hkg-mocsgw-01',
 'hkg-sanmgt-01',
 'nyc-sanmgt-01',
 'was-sanmgt-01'}

In [511]:
#train.head()

#Loading all the libraries
import pyPdf
import os
import pandas as pd
import string
import numpy as np
from nltk.corpus import stopwords
import xgboost as xgb
import operator

all_processed_desc = train.processed_desc.append(test.processed_desc)
#all_processed_desc = train1.processed_desc
#creating tfidf 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(#analyzer = "word",   \
                             #tokenizer = None,    \
                             #preprocessor = None, \
                             #stop_words = None,   \
                             max_features = 10000, \
                             ngram_range= (1,2)
                            ) 

train_test_tdidf_features = vectorizer.fit_transform(all_processed_desc)
train_test_tdidf_features = train_test_tdidf_features.toarray()

df_train_test_tfidf_features = pd.DataFrame(train_test_tdidf_features)
df_train_test_tfidf_features.columns = vectorizer.get_feature_names()

df_train_test_tfidf_features = df_train_test_tfidf_features[final_features1]
#train1_tfidf = vectorizer.transform(train1.processed_desc)
#train2_tfidf = vectorizer.transform(train2.processed_desc)

df_train = df_train_test_tfidf_features[0:9850]
df_test = df_train_test_tfidf_features[9850:]

print df_train.shape
print df_test.shape

print train.shape
print test.shape

(9850, 1251)
(21070, 1251)
(9850, 12)
(21070, 13)


In [510]:
final_features = set(all_ent) & set(df_train.columns)
#train[train.StringToExtract=='nyc-cca3zdc-01']
#set(df_train.columns)
final_features1 = list(final_features)
final_features1[0:4]

['hkg_cca4_sf1', 'hkg_cca4_sf2', 'ddl_sys_04', 'esc_evt_02']

In [524]:
from sklearn.svm import SVC, LinearSVC, NuSVC

clf = LinearSVC(C=3)
clf.fit(df_train,train.StringToExtract)
svm_lavel_test =clf.predict(df_test)
svm_lavel_train =clf.predict(df_train)

train_xgb_String = svm_lavel_train
test_xgb_String = svm_lavel_test

In [534]:
train['xgb_String'] = train_xgb_String
test['xgb_String'] = test_xgb_String

In [547]:
#train[train.is_ip==1].shape
train[(train.StringToExtract<>train.xgb_String) &  (train.is_ip==0) ].StringToExtract.value_counts()

ccsearch      14
knowledge     13
knowwho        9
internal       3
will           3
hkg-mb-14      2
ddl-ch-v03     1
dtype: int64

In [631]:
#train[train.StringToExtract=='ddl-ch-v03']

In [594]:
test[test.final_string=='ddl-ch-v03']

,id,description,processed_desc,present_ip,ddl_0_keywords,ddl_1_keywords,ddl_2_keywords,ddl_3_keywords,xgb_String,best_sub_String,is_ip,others,final_string


In [ ]:
## Manual correction -- ID - 25398 with time
## 8726 - will

In [560]:
print train[train.StringToExtract<>train.xgb_String].shape
print test[(test.final_string==test.xgb_String) | (test.is_ip==1)].shape
print test.shape

print 100*20898.0/test.shape[0]

(130, 12)
(20898, 13)
(21070, 13)
99.1836734694


In [587]:
#test[test.xgb_String=='will']

In [ ]:
#train['StringToExtract_final'] = train.apply(extract_string,axis=1)
test['StringToExtract_final'] = test.apply(extract_string,axis=1)
print sum(np.where(train.StringToExtract==train.StringToExtract_final,0,1))
train['error'] = np.where(train.StringToExtract==train.StringToExtract_final,0,1)

In [ ]:
#test[test.StringToExtract_final=='hkg-ev-14']

In [ ]:
samplesubmission.StringToExtract = test.StringToExtract_final
samplesubmission.to_csv('SVM_with_Precedence_v9.csv',index=False)
samplesubmission.head(10)

In [ ]:
## Post Analysis

print test[test.ddl_1_keywords<>test.StringToExtract_final].shape[0]
print test.shape[0]

100*(20769.0/21070)

In [ ]:
ddl_not_as_final = test[(test.ddl_1_keywords<>test.StringToExtract_final) & 
                        (test.present_ip<>test.StringToExtract_final) & (test.StringToExtract_final<>'service') ]
print ddl_not_as_final.shape
print 100*(1-57.00/21070)

In [ ]:
#ddl_not_as_final.StringToExtract_final.value_counts()
#100*(57.00/21070) + 99.473
#test.StringToExtract_final.value_counts()
100*(43.0/21070)+99.473

In [ ]:
test['Pre_final_String'] = np.where(((test.ddl_1_keywords==test.StringToExtract_final) |
                        (test.present_ip==test.StringToExtract_final) |
                                    (test.StringToExtract_final<>'service')),test.StringToExtract_final,'not matched')

In [ ]:
samplesubmission.StringToExtract = test.Pre_final_String
samplesubmission.head(10)
samplesubmission.to_csv('Jeeban_XGB_with_Precedence_v8.csv',index=False)

In [ ]:
#test.Pre_final_String.value_counts()
99.473

In [ ]:
#samplesubmission[samplesubmission.StringToExtract=='psha2532']

In [ ]:
train.to_csv('featured_train.csv',index=False)

In [ ]:
test_error = test[test.StringToExtract_final<>test.ddl_1_keywords]
train_error = train[train.StringToExtract<>train.ddl_1_keywords]

In [ ]:
#train_error

In [ ]:
train1 = train.sample(frac=0.7)
train2 = train[~train.index.isin (train1.index)]
print train1.shape
print train2.shape
print train1.is_matched.mean()
print train2.is_matched.mean()

In [449]:
#help(CountVectorizer)
#samp = 'my new text is like this love\\I want to do'
#vectorizer.fit_transform(samp)
corpus = [
     'This is the first document.',
     'This is the second sec.ond document.',
     'And the third one.',
     'Is this the first document?',
 ]
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names()

[u'and',
 u'document',
 u'first',
 u'is',
 u'ond',
 u'one',
 u'sec',
 u'second',
 u'the',
 u'third',
 u'this']

In [ ]:
#train.head()

#Loading all the libraries
import pyPdf
import os
import pandas as pd
import string
import numpy as np
from nltk.corpus import stopwords
import xgboost as xgb
import operator

all_processed_desc = train.processed_desc.append(test.processed_desc)
#all_processed_desc = train1.processed_desc
#creating tfidf 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             #max_features = 10000, \
                             ngram_range= (1,2)
                            ) 

train_test_tdidf_features = vectorizer.fit_transform(all_processed_desc)
train_test_tdidf_features = train_test_tdidf_features.toarray()

df_train_test_tfidf_features = pd.DataFrame(train_test_tdidf_features)
df_train_test_tfidf_features.columns = vectorizer.get_feature_names()

train1_tfidf = vectorizer.transform(train1.processed_desc)
train2_tfidf = vectorizer.transform(train2.processed_desc)

#df_train = df_train_test_tfidf_features[0:9850]
#df_test = df_train_test_tfidf_features[9850:]

#print df_train.shape
#print df_test.shape

print train.shape
print test.shape

In [ ]:
#imp_features = df.feature
#d_train = xgb.DMatrix(df_train_test_tfidf_features[imp_features], label=train.String_Class) ## 2nd run imp features
d_train = xgb.DMatrix(train1_tfidf, label=train1.is_ip)

params = {}
#params['objective'] = 'binary:logistic'
params['objective'] = 'multi:softmax'
#params['objective'] = 'multi:softprob'
params['eval_metric'] = 'merror'
params['eta'] = 0.2
params['max_depth'] = 10
params["subsample"] = 0.7
params['num_class'] = 2 # 12 Classes
params["min_child_weight"] = 0.01
params["colsample_bytree"] = 0.8
#model = xgb.cv(dtrain=d_train,nfold=6,params=params,num_boost_round=200)
model = xgb.train(dtrain=d_train,params=params,num_boost_round=10)

In [ ]:
def extract_string(row):
    if(row.xgb_String==0):
        return row.present_ip
    
    elif(row.xgb_String==1):
        return 'service'
    
    elif(row.xgb_String==2):
        return 'internal'
    
    elif(row.xgb_String==3):
        return 'will'
    
    elif(row.xgb_String==4):
        return 'author'
    
    elif(row.xgb_String==5):
        return 'error'
    
    elif(row.xgb_String==6):
        return 'connect'
    
    elif(row.xgb_String==7):
        return 'onlineservices'
    
    elif(row.xgb_String==8):
        return 'hkg-ev-14'
    
    elif(row.xgb_String==9):
        return 'ddl-dc-03'
    
    elif(row.xgb_String==10):
        return 'hkg-mb-14'
    
    
    elif(row.xgb_String==11):
        return 'ddl-bnt-01'
    
    elif(row.xgb_String==12):
        return 'hkg-dcr-02'
    
    elif(row.xgb_String==13):
        return 'au-per-07a-stap-01'
    
    elif(row.xgb_String==14):
        return 'time' 
    
    elif(row.xgb_String==15):
        return 'hdr-mb-16'
    
    elif(row.xgb_String==16):
        return 'bru-dc-02'
    
    elif(row.xgb_String==17):
        return 'dub-dc-03'
    
    elif(row.xgb_String==18):
        return 'dc=internal'
    
    elif(row.xgb_String==19):
        return 'esc-mb-03'
    
    elif(row.xgb_String==20):
        return 'esc-mb-04'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def get_final_class(x):
    if(x=='hkg-dcr-02'):
        return 1
    else:
        return 0
train1['final_class'] = train1.StringToExtract.apply(get_final_class)
train2['final_class'] = train2.StringToExtract.apply(get_final_class)
print train1['final_class'].sum()
print train2['final_class'].sum()

from sklearn.svm import SVC, LinearSVC, NuSVC

clf = LinearSVC(C=0.01)
clf.fit(train1_tfidf,train1.final_class)
svm_lavel_test =clf.predict(train2_tfidf)
train2_xgb_String = svm_lavel_test
train2_xgb_String
train2['predicted_final_class'] = train2_xgb_String

print train2[train2.final_class<>train2.predicted_final_class].shape[0]
print train2.shape[0]


In [ ]:
#train2[train2.final_class<>train2.predicted_final_class]
#train[train.StringToExtract=='hkg-ev-14'].shape
test[(test.ddl_1_keywords<>test.StringToExtract_final)].shape

In [ ]:
100*301.0/test.shape[0]

In [ ]:
test[test.StringToExtract_final=='hkg-ev-14'].shape

In [ ]:
#will = 14
#eternal = 7
#hkg-ev-14 = 7
#ddl-dc-03 = 7
#hkg-dcr-02 =7
#hdr-mb-16 =7


In [ ]:
#train2[train2.predicted_final_class==0].shape
#test.columns

In [ ]:
print train[train.StringToExtract=='internal'].shape
print test[test.StringToExtract_final=='internal'].shape

In [ ]:
#test[test.StringToExtract_final=='internal']

In [ ]:
print test[test.ddl_1_keywords=='will'].shape
print train[train.ddl_1_keywords=='will'].shape

#test.shape[0]*5.0/9850

In [ ]:
test.shape[0]*1.0/2955

In [ ]:
from sklearn.svm import SVC, LinearSVC, NuSVC

clf = LinearSVC(C=1)
clf.fit(df_train,train.is_ip)
svm_lavel_test =clf.predict(df_test)
test['predicted_ip'] = svm_lavel_test

In [ ]:
#test.shape

In [ ]:
#train[train.is_ip==1].shape
120.0/9850

In [ ]:
print test[(test.xgb_String==0)].shape
print test[(test.predicted_ip==1) & (test.xgb_String==0)].shape

In [ ]:
100*201.0/21070

In [ ]:
#print train1.is_ip.value_counts()
#print train2.is_ip.value_counts()
#train2.shape[0]*0.000725

train1_xgb = xgb.DMatrix(train1_tfidf)
train1_xgb_String = np.array(model.predict(train1_xgb),dtype='int').tolist()
train1['predicted_ip'] = train1_xgb_String

train2_xgb = xgb.DMatrix(train2_tfidf)
train2_xgb_String = np.array(model.predict(train2_xgb),dtype='int').tolist()
train2['predicted_ip'] = train2_xgb_String


#train1_xgb_String

In [ ]:
print train2[train2.is_ip<>train2.predicted_ip].shape[0]
print train2.shape[0]
print test.shape[0]*(4.0/2955)
print 100*(4.0/2955)

In [ ]:
#print train2.is_ip.sum()
#print train2.shape
#print 100*34/2955.0

In [ ]:
#train2[train2.is_ip==1]
#train2.head()

In [ ]:
#train[train.StringToExtract=='will']
#test.predicted_ip.sum()

In [ ]:
#test.head()

In [ ]:
test['final_String'] = test.ddl_1_keywords

In [ ]:
test['final_String'] = np.where(test.predicted_ip==1,test.StringToExtract_final,test.final_String)

In [ ]:
test['final_String'] = np.where(test.StringToExtract_final=='service',test.StringToExtract_final,test.final_String)

In [ ]:
test['final_String'] = np.where(test.StringToExtract_final==test.final_String,test.final_String,'to be matched')

In [ ]:
#test[test.StringToExtract_final<>'to be matched']

In [ ]:
100*57.0/test.shape[0]

In [ ]:
#test[test.final_String=='to be matched']
100*15.0/test.shape[0]
99.76+0.071

In [359]:
test.columns

Index([u'id', u'description', u'processed_desc', u'present_ip',
       u'ddl_0_keywords', u'ddl_1_keywords', u'ddl_2_keywords',
       u'ddl_3_keywords', u'best_sub_String', u'is_ip', u'others',
       u'final_string'],
      dtype='object')

In [632]:
samplesubmission.StringToExtract = test.final_string
#samplesubmission.StringToExtract 
samplesubmission.to_csv('final_version_v_new.csv',index=False)
samplesubmission.head(10)

,id,StringToExtract
0,1,frapub2
1,2,lpap02z1
2,4,ddlsql43
3,5,ddl-ubs-28-ca-asw1
4,6,ddlsql43
5,8,ddlsql43
6,9,frapub2
7,10,ddlsql43
8,11,esc-spr4web-01
9,12,ddlsql43


In [ ]:
print error.StringToExtract.value_counts()

In [ ]:
error_sample = train[(train.is_matched==0) & (train.is_ip==0) & (train.StringToExtract<>'service') & (train.StringToExtract<>'will')]
print error_sample.StringToExtract.value_counts()
print train[train.StringToExtract=='hkg-ev-14'].shape

In [ ]:
#train[train.StringToExtract=='will']
train_will = train[train.StringToExtract=='hkg-ev-14']
train_not_will = train[train.StringToExtract<>'hkg-ev-14']
all_will_string = set(train_will.processed_desc.str.cat(sep=' ').split(' '))
all_not_will_string = set(train_not_will.processed_desc.str.cat(sep=' ').split(' '))
unq_word = all_will_string - all_not_will_string
unq_word

In [786]:
train_will = train[train.is_ip==1]
train_not_will = train[train.is_ip<>1]
all_will_string = set(train_will.processed_desc.str.cat(sep=' ').split(' '))
all_not_will_string = set(train_not_will.processed_desc.str.cat(sep=' ').split(' '))
unq_word = all_will_string - all_not_will_string
unq_word

{'\r\r\nnode',
 '***************',
 '0\r\r\nworking',
 '1\r\r\n\r\r\n',
 '10809',
 '11090',
 '11106',
 '11896',
 '12309',
 '131399',
 '131400',
 '132093',
 '14401',
 '146)',
 '15-signaling',
 '17071',
 '18219',
 '19187',
 '196\\cisco',
 '19932',
 '2017,',
 '20837',
 '2114x',
 '21882',
 '21992',
 '221799',
 '2260',
 '22717',
 '228)',
 '23646',
 '24329',
 '2472',
 '25)',
 '25143',
 '252)',
 '26)',
 '2615037',
 '2656057',
 '27)',
 '27243',
 '3376',
 '390',
 '403144',
 '409400',
 '41\\1352\\\r\r\n\r\r\n\r\r\none',
 '455',
 '50\r\r\n\r\r\n',
 '50%',
 '500',
 '500\r\r\n\r\r\n',
 '612886',
 '615592',
 '632695',
 '634495',
 '649797',
 '657885',
 '658793',
 '66214',
 '689387',
 '68\\process',
 '70)',
 '7221',
 '8659',
 '89)',
 '91)',
 '96)',
 '\\\\10',
 'admin,connection',
 'administration,cisco',
 'agentminrequiredversioncheck',
 'api,cisco',
 'backup=pper515',
 'backup=prom1217',
 'bei-fw-01',
 'blr-btp-ml-stfp-01',
 'blr-btp-ml-stfp-51',
 'bper810',
 'bw',
 'bxsp911',
 'cas-fw-01',
 'ccx',
 

In [ ]:
train[train.StringToExtract=='hkg-ev-14']

In [ ]:
def find_internal(row):
    will_index1 = row.description.find('reenter')
    #will_index2 = row.description.find('alert on event id 22')
    #will_index2 = row.description.find(' 60% ')
    will_index2 = row.description.find(' dns ')
    
    #if((will_index1<>-1) or (will_index2<>-1) or (will_index3<>-1)):
    if((will_index1<>-1) & (will_index2<>-1)):
    #if((will_index1<>-1)):
        return 'internal'
    else:
        return row.ddl_1_keywords

train['is_internal'] = train.apply(find_internal,axis=1)   
test['is_internal'] = test.apply(find_internal,axis=1)

In [ ]:
#test[test.is_internal=='internal']

In [ ]:
final_submission = pd.read_csv('SVM_with_Precedence_v9_will_attached.csv')

In [ ]:
final_submission[final_submission['id'].isin(test[test.is_internal=='internal'].id)

In [ ]:
final_submission['StringToExtract'] = np.where(final_submission['id'].isin(test[test.is_internal=='internal'].id),'internal',final_submission['StringToExtract'])

In [ ]:
final_submission[final_submission.StringToExtract=='internal'].shape

In [ ]:
final_submission.to_csv('regex_will_internal_twaeked_1.csv',index=False)

In [644]:
train_entity_score = pd.DataFrame(100*train.StringToExtract.value_counts()/len(train)).reset_index()
test_entity_score = pd.DataFrame(100*test.final_string.value_counts()/len(test)).reset_index()

In [670]:
train_test_entity_score =pd.merge(train_entity_score,test_entity_score,how='left',on='index',left_index=True)
train_test_entity_score['score_diff'] = abs(train_test_entity_score['0_x']- train_test_entity_score['0_y'])
train_test_entity_score.sort(columns=['score_diff'],ascending=False,inplace=True)


,index,0_x,0_y,score_diff
0,ddlsql43,38.751269,42.263882,3.512613
1,esc-ev-11,4.487310,4.893213,0.405903
2,nycpub,1.167513,1.262458,0.094946
3,ddlsql37,1.106599,1.196013,0.089414
4,ddl-mocsrds-01,1.065990,1.153299,0.087309


In [697]:
#train_test_entity_score[0:100]

not_in_test = list(set(train.StringToExtract.unique()) - set(test.final_string.unique()))
print (len(not_in_test))
train_not_in_test = train[train.StringToExtract.isin(not_in_test)]
#train_not_in_test
#print test[test.xgb_String=='ddlsql43'].shape
#print test[test.ddl_1_keywords=='ddlsql43'].shape

99
(8905, 13)
(8905, 13)


In [713]:
errored_pattern_0("monitoring failed to run the test-mrsteahhl cmdlet. an active directory error 0x51 occurred when trying to check the suitability of server ddl-dc-06.internal.ecnahcdroffilc.net.")

'ddl-dc-06'

In [ ]:
## chk id 87,124,2707,2856,3266,8242,16109
## 13071,14683,14981,5957,14683,14981,16109

In [720]:
mismatch.shape

(203, 13)

In [748]:
to_be_checked =test[test['id'].isin([87,178,1559,3935,4131,4142,4738,5743,5898,6173,8168,
                                    8722,8726,10914,10930,12315,13789,14077,17429,19132,
                                    19287,20010,21140,21473,21903,21972,22879,23554,23931,
                                    25332,27702,30058,])]
to_be_checked

(32, 13)

In [755]:
test.shape[0] -0.9983*test.shape[0]


35.818999999999505

In [554]:
#to_be_checked[['id','description','ddl_1_keywords','xgb_String','final_string']]

In [193]:
#train[train.StringToExtract=='atlas']

In [746]:
#to_be_checked[['description','ddl_1_keywords','xgb_String','final_string']]

In [758]:
#mismatch = test[(test.ddl_1_keywords<>test.xgb_String) & (test.is_ip==0)]
#mismatch[['id','description','ddl_1_keywords','xgb_String','final_string']]

In [759]:
#train[train.StringToExtract=='fra-dc-03'].head()